In [49]:
#first import mikes custom function woooooooot!!!
#create fetchall function
import sqlite3
conn = sqlite3.connect('jobs.db') #set db conncetion
def fetchall(q_x):
    names = [] #initialize list of col names
    [names.append(name[0]) for name in conn.execute(q_x).description] #extract column names with description method
    print(names,'\n')
    for row in conn.execute(q_x).fetchall():
        print(row)

fetchall('Select * from recent_grads')

['index', 'Rank', 'Major_code', 'Major', 'Major_category', 'Total', 'Sample_size', 'Men', 'Women', 'ShareWomen', 'Employed', 'Full_time', 'Part_time', 'Full_time_year_round', 'Unemployed', 'Unemployment_rate', 'Median', 'P25th', 'P75th', 'College_jobs', 'Non_college_jobs', 'Low_wage_jobs'] 

(0, 1, 2419, 'PETROLEUM ENGINEERING', 'Engineering', 2339, 36, 2057, 282, 0.120564344, 1976, 1849, 270, 1207, 37, 0.018380527, 110000, 95000, 125000, 1534, 364, 193)
(1, 2, 2416, 'MINING AND MINERAL ENGINEERING', 'Engineering', 756, 7, 679, 77, 0.10185185199999999, 640, 556, 170, 388, 85, 0.117241379, 75000, 55000, 90000, 350, 257, 50)
(2, 3, 2415, 'METALLURGICAL ENGINEERING', 'Engineering', 856, 3, 725, 131, 0.153037383, 648, 558, 133, 340, 16, 0.024096386, 73000, 50000, 105000, 456, 176, 0)
(3, 4, 2417, 'NAVAL ARCHITECTURE AND MARINE ENGINEERING', 'Engineering', 1258, 16, 1123, 135, 0.107313196, 758, 1069, 150, 692, 40, 0.050125313, 70000, 43000, 80000, 529, 102, 0)
(4, 5, 2405, 'CHEMICAL ENGINEE

# 1. Intro

In the last mission, we computed summary statistics across columns with SQL. In many cases, though, we want to drill down even more and compute summary statistics per group. In this mission, we'll explore how to calculate more granular summary statistics. We'll switch back to writing SQL queries directly instead of using Python so we can focus on the SQL syntax.

We'll continue to work with the American Community Survey data on college majors and job outcomes. Here's a preview of recent-grads.csv, the data set we'll be working with:
![](Picture3.png)

We've loaded a subset of the data into a table named recent_grads in a database. The subset contains the 2010-2012 data for recent college grads only. The full table has many more columns (21 to be specific) than the ones displayed above. You can find more details on them at FiveThirtyEight's Github repo.

Here are the descriptions for the columns in the preview:

- Rank - The major's rank by median earnings
- Major_code - The major's code or ID
- Major - The name of the major
- Major_category - The broader category the major belongs to
- Total - The total number of people who studied the major
- Sample_size - The sample size (unweighted) of graduates with full time jobs
- Men - The number of male graduates
- Women - The number of female graduates
- ShareWomen - Women as a proportion of the total number of graduates (a number ranging from 0 to 1)
- Employed - The number of employed graduates

As we progress through this mission, we'll drill down and compute summary statistics by group to answer questions like:

- What's the share of women in each major category?
- Which major categories have the greatest numbers of employed graduates?
- What percentage of people in each major category end up in low-wage jobs?

First, let's explore the data.

# 2. Intro (again I guess...)

Write a SQL query that displays all of the columns and the first five rows of the recent_grads table.

In [50]:
import sqlite3 as sql

conn = sql.connect('jobs.db')
q = 'SELECT * from recent_grads LIMIT 5;'
data = conn.execute(q).fetchall()

data

[(0,
  1,
  2419,
  'PETROLEUM ENGINEERING',
  'Engineering',
  2339,
  36,
  2057,
  282,
  0.120564344,
  1976,
  1849,
  270,
  1207,
  37,
  0.018380527,
  110000,
  95000,
  125000,
  1534,
  364,
  193),
 (1,
  2,
  2416,
  'MINING AND MINERAL ENGINEERING',
  'Engineering',
  756,
  7,
  679,
  77,
  0.10185185199999999,
  640,
  556,
  170,
  388,
  85,
  0.117241379,
  75000,
  55000,
  90000,
  350,
  257,
  50),
 (2,
  3,
  2415,
  'METALLURGICAL ENGINEERING',
  'Engineering',
  856,
  3,
  725,
  131,
  0.153037383,
  648,
  558,
  133,
  340,
  16,
  0.024096386,
  73000,
  50000,
  105000,
  456,
  176,
  0),
 (3,
  4,
  2417,
  'NAVAL ARCHITECTURE AND MARINE ENGINEERING',
  'Engineering',
  1258,
  16,
  1123,
  135,
  0.107313196,
  758,
  1069,
  150,
  692,
  40,
  0.050125313,
  70000,
  43000,
  80000,
  529,
  102,
  0),
 (4,
  5,
  2405,
  'CHEMICAL ENGINEERING',
  'Engineering',
  32260,
  289,
  21239,
  11021,
  0.341630502,
  25694,
  23170,
  5180,
  16697,
  

In [51]:
conn.execute('PRAGMA TABLE_INFO(recent_grads);').fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'Rank', 'INTEGER', 0, None, 0),
 (2, 'Major_code', 'INTEGER', 0, None, 0),
 (3, 'Major', 'TEXT', 0, None, 0),
 (4, 'Major_category', 'TEXT', 0, None, 0),
 (5, 'Total', 'INTEGER', 0, None, 0),
 (6, 'Sample_size', 'INTEGER', 0, None, 0),
 (7, 'Men', 'INTEGER', 0, None, 0),
 (8, 'Women', 'INTEGER', 0, None, 0),
 (9, 'ShareWomen', 'REAL', 0, None, 0),
 (10, 'Employed', 'INTEGER', 0, None, 0),
 (11, 'Full_time', 'INTEGER', 0, None, 0),
 (12, 'Part_time', 'INTEGER', 0, None, 0),
 (13, 'Full_time_year_round', 'INTEGER', 0, None, 0),
 (14, 'Unemployed', 'INTEGER', 0, None, 0),
 (15, 'Unemployment_rate', 'REAL', 0, None, 0),
 (16, 'Median', 'INTEGER', 0, None, 0),
 (17, 'P25th', 'INTEGER', 0, None, 0),
 (18, 'P75th', 'INTEGER', 0, None, 0),
 (19, 'College_jobs', 'INTEGER', 0, None, 0),
 (20, 'Non_college_jobs', 'INTEGER', 0, None, 0),
 (21, 'Low_wage_jobs', 'INTEGER', 0, None, 0)]

# 3. Calculating Group-Level Summary Statistics

The GROUP BY SQL statement allows us to compute summary statistics by "group," or unique value. When we use this statement, SQL creates a group for each unique value in a column or set of columns (the same values we get when we use the DISTINCT statement), and then does the calculations for them. To illustrate, we can find the total number of people employed in each major category with the following query:

    SELECT SUM(Employed) 
    FROM recent_grads 
    GROUP BY Major_category;
    
This will give us the total number of employed graduates for each major category. Here's a truncated view of the output:

    SUM(Employed)
    66943
    288114
    302797
    
The output shows aggregate counts of the Employed column for each Major_category. Unfortunately, it doesn't indicate which major category each row refers to. We can fix this by including the Major_category column in our query:

    SELECT Major_category, SUM(Employed) 
    FROM recent_grads 
    GROUP BY Major_category;
This makes the output much easier to understand:

![](Picture4.png)

Here's how the query works. The GROUP BY statement splits the Major_category column into groups (with one group for each unique major category), then calculates the sum for each group. The following diagram shows how GROUP BY splits the data. (The diagram uses a small sample from the recent_grads table.):

![](Picture5.png)

For each group, the GROUP BY statement queries each column, and runs all of the aggregation functions we include in the query after the SELECT statement:

![](Picture6.png)

If a column is selected, the SQL engine will use the last value for that column in the group. If an aggregation function is selected, the SQL engine will compute the value for that aggregation function across the group.

The query in the diagram will give us the following result:

![](Picture7.png)

In [52]:
sql = """SELECT
        Major_category, Major, SUM(Employed), employed
        FROM recent_grads
        GROUP BY Major_category"""

fetchall(sql)

['Major_category', 'Major', 'SUM(Employed)', 'Employed'] 

('Agriculture & Natural Resources', 'MISCELLANEOUS AGRICULTURE', 66943, 1290)
('Arts', 'DRAMA AND THEATER ARTS', 288114, 36165)
('Biology & Life Science', 'ZOOLOGY', 302797, 6259)
('Business', 'HOSPITALITY MANAGEMENT', 1088742, 36728)
('Communications & Journalism', 'MASS MEDIA', 330660, 44679)
('Computers & Mathematics', 'COMMUNICATION TECHNOLOGIES', 237894, 14779)
('Education', 'LIBRARY SCIENCE', 479839, 742)
('Engineering', 'MECHANICAL ENGINEERING RELATED TECHNOLOGIES', 420372, 4186)
('Health', 'COMMUNICATION DISORDERS SCIENCES AND SERVICES', 372147, 29763)
('Humanities & Liberal Arts', 'COMPOSITION AND RHETORIC', 544118, 15053)
('Industrial Arts & Consumer Services', 'COSMETOLOGY SERVICES AND CULINARY ARTS', 189043, 8650)
('Interdisciplinary', 'MULTI/INTERDISCIPLINARY STUDIES', 9821, 9821)
('Law & Public Policy', 'CRIMINAL JUSTICE AND FIRE PROTECTION', 144790, 125393)
('Physical Sciences', 'ATMOSPHERIC SCIENCES AND METEOROL

In [53]:
sql = """SELECT COUNT(DISTINCT Major_category) FROM recent_grads"""
fetchall(sql)

['COUNT(DISTINCT Major_category)'] 

(16,)


# 4. Calculating Group-Level Summary Statistics

- Use the SELECT statement to select the following columns and aggregates in a query:
    - Major_category
    - AVG(ShareWomen)
- Use the GROUP BY statement to group the query by the Major_category column.

In [54]:
sql='''SELECT Major_category, AVG(ShareWomen) 
FROM recent_grads 
GROUP BY Major_category;'''

fetchall(sql)

['Major_category', 'AVG(ShareWomen)'] 

('Agriculture & Natural Resources', 0.6179384232)
('Arts', 0.56185119575)
('Biology & Life Science', 0.584518475857143)
('Business', 0.4050631853076923)
('Communications & Journalism', 0.64383484025)
('Computers & Mathematics', 0.5127519954545455)
('Education', 0.6749855163125)
('Engineering', 0.2571578951034483)
('Health', 0.6168565694166667)
('Humanities & Liberal Arts', 0.6761934042)
('Industrial Arts & Consumer Services', 0.4493512688571429)
('Interdisciplinary', 0.495397153)
('Law & Public Policy', 0.3359896912)
('Physical Sciences', 0.5087494197)
('Psychology & Social Work', 0.7777631628888888)
('Social Science', 0.5390672957777778)


# 5. Renaming Columns With the AS Statement

You may have noticed that on the last screen, specifying AVG(ShareWomen) caused the column to have that name in the results. This can lead to confusion, and make it difficult to work with the results of SQL queries. To avoid this issue, we can select and rename columns at the same time with the AS statement. Here's an example:

    SELECT AVG(ShareWomen) AS average_female_share 
    FROM recent_grads;

This query will result in the following output:

    average_female_share
    0.5225502029537575
    
**Instructions**
- Write a query that selects the following items, in order, and renames them with AS:
    - SUM(Men) as total_men.
    - SUM(Women) as total_women.

In [55]:
sql = '''SELECT SUM(Men) AS total_men, SUM(Women) AS total_women 
FROM recent_grads;'''

fetchall(sql)

['total_men', 'total_women'] 

(2878263, 3897752)


# 6. Practice - Using GROUP BY

Now that we have a better understanding of the GROUP BY statement, let's practice using it by computing summary statistics by group for the recent_grads table.

**Instructions**

- For each major category, find the percentage of graduates who are employed.
    - Use the SELECT statement to select the following columns and aggregates in your query:
        - Major_category
        - AVG(Employed) / AVG(Total) as share_employed
    - Use the GROUP BY statement to group the query by the Major_category column.

In [56]:
q = '''SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
FROM recent_grads 
GROUP BY Major_category
ORDER BY Major_category DESC;'''

conn.execute(q).fetchall()

[('Social Science', 0.7575825619001975),
 ('Psychology & Social Work', 0.790724459311403),
 ('Physical Sciences', 0.7506564085422069),
 ('Law & Public Policy', 0.8083994483744353),
 ('Interdisciplinary', 0.7987150292778139),
 ('Industrial Arts & Consumer Services', 0.8226700668430581),
 ('Humanities & Liberal Arts', 0.7626382682895378),
 ('Health', 0.8033741337996244),
 ('Engineering', 0.7819666916550562),
 ('Education', 0.858190149321534),
 ('Computers & Mathematics', 0.7956108197773972),
 ('Communications & Journalism', 0.8422291333949735),
 ('Business', 0.8359659576036412),
 ('Biology & Life Science', 0.6671565365683841),
 ('Arts', 0.8067482429367457),
 ('Agriculture & Natural Resources', 0.8369862842425075)]

In [57]:
sql = """SELECT 
        Major_category, 
        ROUND(AVG(ShareWomen),2) AS Share_Women
        FROM recent_grads 
        GROUP BY Major_category
        HAVING Share_Women > 0.5
        ORDER BY Share_Women DESC"""
fetchall(sql)

['Major_category', 'Share_Women'] 

('Psychology & Social Work', 0.78)
('Humanities & Liberal Arts', 0.68)
('Education', 0.67)
('Communications & Journalism', 0.64)
('Agriculture & Natural Resources', 0.62)
('Health', 0.62)
('Biology & Life Science', 0.58)
('Arts', 0.56)
('Social Science', 0.54)
('Computers & Mathematics', 0.51)
('Physical Sciences', 0.51)


# 7. Querying Virtual Columns with the HAVING statement

Sometimes we want to select a subset of rows after performing a GROUP BY query. On the last screen, for instance, we may have wanted to select only those rows where share_employed is greater than .8. We can't use the WHERE clause to do this because share_employed isn't a column in recent_grads; it's actually a virtual column generated by the GROUP BY statement.

When we want to filter on a column generated by a query, we can use the HAVING statement. Here's an example:

    SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
    FROM recent_grads 
    GROUP BY Major_category 
    HAVING share_employed > .8;

Note that we used the same column name in the HAVING statement that we originally specified with the AS statement. SQL allows us to use custom column names in subsequent statements, including HAVING and WHERE. The statement above will result in the following output:

![](Picture8.png)

Note that the results only include categories where share_employed is greater than .8. That's because the HAVING statement filters out the other rows.

**Instructions**

- Find all of the major categories where the share of graduates with low-wage jobs is greater than .1.
    - Use the SELECT statement to select the following columns and aggregates in a query:
        - Major_category
        - AVG(Low_wage_jobs) / AVG(Total) as share_low_wage
    - Use the GROUP BY statement to group the query by the Major_category column.
    - Use the HAVING statement to restrict the selection to rows where share_low_wage is greater than .1.

In [58]:
q = '''SELECT Major_category, AVG(Low_wage_jobs) / AVG(Total) AS share_low_wage 
FROM recent_grads 
GROUP BY Major_category 
HAVING share_low_wage > .1;'''

fetchall(q)


['Major_category', 'share_low_wage'] 

('Arts', 0.16833085991095678)
('Communications & Journalism', 0.1263241815481876)
('Humanities & Liberal Arts', 0.13208721344194835)
('Industrial Arts & Consumer Services', 0.11571334076033978)
('Law & Public Policy', 0.11568503743572278)
('Psychology & Social Work', 0.11693384919554187)
('Social Science', 0.10223297343603174)


In [59]:
q = '''SELECT Major_category, ROUND(AVG(Low_wage_jobs) / AVG(Total),3) AS share_low_wage 
FROM recent_grads 
GROUP BY Major_category 
HAVING share_low_wage > .1;'''

fetchall(q)


['Major_category', 'share_low_wage'] 

('Arts', 0.168)
('Communications & Journalism', 0.126)
('Humanities & Liberal Arts', 0.132)
('Industrial Arts & Consumer Services', 0.116)
('Law & Public Policy', 0.116)
('Psychology & Social Work', 0.117)
('Social Science', 0.102)


# 8. Rounding Results with the ROUND function

On the last screen, the percentages in our results were very long and hard to read (e.g., 0.16833085991095678). We can use the SQL ROUND function in our query to round them. Here's an example of what this looks like:

    SELECT Major_category, ROUND(ShareWomen, 2) AS rounded_share_women 
    FROM recent_grads;
    

The query will round the ShareWomen column to two decimal places. Here's a truncated view of the results:

![](Capture.png)

By passing different values in to the ROUND function, such as ROUND(ShareWomen, 3), we can round to different decimal places.

**Instructions**

- Write a SQL query that returns the following columns of recent_grads (in the same order):
    - ShareWomen rounded to 4 decimal places
    - Major_category
- Limit the results to 10 rows.

In [60]:
sql='''SELECT ROUND(ShareWomen, 2) AS Share_Women, Major_category 
FROM recent_grads 
LIMIT 10;'''

fetchall(sql)

['Share_Women', 'Major_category'] 

(0.12, 'Engineering')
(0.1, 'Engineering')
(0.15, 'Engineering')
(0.11, 'Engineering')
(0.34, 'Engineering')
(0.14, 'Engineering')
(0.54, 'Business')
(0.44, 'Physical Sciences')
(0.14, 'Engineering')
(0.44, 'Engineering')


# 9. Nesting Functions

On a previous screen, we ran the following query:

    SELECT Major_category, AVG(Employed) / AVG(Total) AS share_employed 
    FROM recent_grads 
    GROUP BY Major_category 
    HAVING share_employed > .8;
This query returned very long fractional values for share_employed. We can update our query with the ROUND function to round the results to three decimal places:

    SELECT Major_category, ROUND(AVG(Employed) / AVG(Total), 3) AS share_employed 
    FROM recent_grads 
    GROUP BY Major_category 
    HAVING share_employed > .8;
This will return the following result:

![](Capture1.png)

**Instructions**
- Use the SELECT statement to select the following columns and aggregates in a query:
    - Major_category
    - AVG(College_jobs) / AVG(Total) as share_degree_jobs
        - Use the ROUND function to round share_degree_jobs to 3 decimal places.
- Group the query by the Major_category column.
- Only select rows where share_degree_jobs is less than .3.

In [61]:
q='''SELECT 
    Major_category, 
    ROUND(AVG(College_jobs) / AVG(Total), 3) AS share_degree_jobs 
    FROM recent_grads 
    GROUP BY Major_category 
    HAVING share_degree_jobs < .3;'''

fetchall(q)

['Major_category', 'share_degree_jobs'] 

('Agriculture & Natural Resources', 0.248)
('Arts', 0.265)
('Business', 0.114)
('Communications & Journalism', 0.22)
('Humanities & Liberal Arts', 0.27)
('Industrial Arts & Consumer Services', 0.249)
('Law & Public Policy', 0.163)
('Social Science', 0.215)


# 10. Next Steps

In this mission, we covered the GROUP BY and HAVING statements. We can use these statements to quickly calculate powerful summary statistics in SQL. In the next few missions, we'll learn more about working with SQL tables, including how to insert and modify data.